# Chapter 3 Solving Non-invertible Linear System

Let us solve $A x = b$ when $A$ is non-square or not of full rank. We will use $LU$-decomposition.

In [1]:
# numerical and scientific computing libraries  
import numpy as np 
import scipy as sp

# plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

In [3]:
# create a random matrix of size m x n with the rank <= k <= min(m, n).
def create_random_matrix(m: int, n: int, k: int) -> np.ndarray:
    if k > min(m, n):
        raise ValueError("k must be less than or equal to min(n, m)")
    A = np.random.randn(m, k)
    B = np.random.randn(k, n)
    return A@B

To find a null space of a matrix $A$, we apply $LU$-decomposition to $A$. Since the null spaces of $A$ and $U$ coincide, we find a null space of the upper triangular matrix $U$. We characterize the pivot entries by searching the first non-zero element in each row of $U$, and then remaining variables are free ones.

In [4]:
def solve_Ax_b(A, b):
    m = A.shape[0]
    n = A.shape[1]
    Aaug = np.hstack((A,-b))
    # LU-decomposition
    P, L, U = sp.linalg.lu(Aaug)
    print(U)

    # find pivot entries
    epsilon = 10**(-10)     # Setting epsilon for negligibles
    pivot = n+1
    rank_u = 0
    pivots = []
    while pivot >= 0:
        non_zero_indices = np.nonzero(U[rank_u,:]**2 > epsilon)
        if np.size(non_zero_indices) == 0 or pivot == non_zero_indices[0][0]:
            pivot = -1
        else:
            pivot = non_zero_indices[0][0]
            pivots.append(pivot)
            rank_u += 1

    print('Rank = ', rank_u)
    print('Pivot variables = ', np.int64(pivots))

    if n in pivots:
        print("The linear system is infeasible since right hand b increases the rank.")
        return
    
    # find column indices of free variables
    free_variables = np.sort(list(set(range(Aaug.shape[1]))- set(pivots)))
    print("Free variables = ", free_variables)

    # Collect pivot columns and free variables
    U_pivot = U[:rank_u, pivots]
    print('pivot columns of U = ')
    print(U_pivot)
    U_free = U[:rank_u, free_variables]
    print('free variable columns of U = ')
    print(U_free)

    # Solve for the pivot part of a basis of the null space of A
    X_pivot = np.linalg.solve(U_pivot,-U_free)
    print('pivot part of X = ')
    print(X_pivot)

    # remove the right hand b from free variables
    free_variables = free_variables[:-1]

    # Combine pivot part and free variable part
    X = np.hstack((np.eye(n)[:,free_variables], np.zeros(n).reshape(-1,1)))
    X[pivots,:] = X_pivot
    print('X = ')
    print(X)

    print(np.allclose(A @ X, np.hstack((np.zeros((m,n-rank_u)),b))))    

In [8]:
# Setting solvability
solvable = 1
# Setting parameters
m=10
n=11
k=7
# generate an mxn matrix of rank at most k
A = create_random_matrix(m, n, k)
if solvable:
    b = A @ np.random.randn(n,1)
else:
    b = np.random.randn(m,1)
    
solve_Ax_b(A,b)

[[-1.8963  1.6811  2.0488 -1.902  -2.6902 -0.5566  1.7623 -0.3202  0.3675 -1.8738 -0.3962  8.4896]
 [ 0.      4.9107 -0.3431 -2.5934 -1.2775 -1.1573  1.081  -2.0346  1.9933 -2.0491 -1.0109  8.6957]
 [ 0.      0.     -3.7365 -0.6424 -3.5679 -4.1025 -4.4956 -2.9216 -0.119  -1.4232  3.3604  2.9159]
 [ 0.      0.      0.     -5.2251 -6.4388 -1.7695 -4.1316 -2.8828  4.2659 -4.0031  1.4477  6.8618]
 [ 0.      0.      0.      0.      2.7695  1.4004  5.3082 -2.7757 -2.4787 -0.7821  0.79   -0.199 ]
 [ 0.      0.      0.      0.      0.     -1.2521  3.0551 -3.8681 -2.9574 -1.6164  1.6879  1.8176]
 [ 0.      0.      0.      0.      0.      0.      2.1399 -1.2648  0.8049 -2.9113  0.8441  6.1059]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.     -0.     -0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.     -0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.     -0.      0.      0.    ]]
Rank =  7